# Generate Data: Gather data, create prompts/payloads of different sizes
---------
*This notebook works best with the conda_python3 kernel on a ml.t3.medium machine*.

### This part of our solution design includes 

- running and downloading our specific dataset

- generating prompts as payloads of different sizes that we will send to our different model endpoints with different combinations of concurrency levels that we will later use to run inference and generate benchmarking metrics and visualizations.

#### This file will generate all data on wikiqa (english version) with prompt sizes 300 - 4000 token lengths in different payload sizes to send to the model endpoint during the inference pipeline. You will also be able to generate the normal wikiqa dataset from the actual 'long bench dataset'. This notebook then focuses on 3 main deliverables:

1. Loading the dataset that is stored within the dataset in the data directory.


2. Generating payloads: This notebook also converts the loaded datasets into payloads based on the input question and records teh context length of the prompt to send as a part of the payload during running inferences on the deployed endpoints.

    - All of the prompts are saved in this data directory in a file named all_prompts.csv.
    

3. Constructing different sized payloads

#### Import all of the necessary libraries below to run this notebook

In [1]:
# if interactive mode is set to no -> pickup fmbench from Python installation path
# if interactive mode is set to yes -> pickup fmbench from the current path (one level above this notebook)
# if interactive mode is not defined -> pickup fmbench from the current path (one level above this notebook)
# the premise is that if run non-interactively then it can only be run through main.py which will set interactive mode to no
import os
import sys
if os.environ.get("INTERACTIVE_MODE_SET", "yes") == "yes":
    sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import io
import copy
import json
import logging
import itertools
import pandas as pd
from fmbench.utils import *
from fmbench.globals import *
from typing import Dict, List

config file current -> configs/config-distilbert-base-uncased.yml, None
Loaded config: {'general': {'name': 'distilbert-base-uncased-v1', 'model_name': 'distilbert-base-uncased'}, 'aws': {'region': 'us-east-1', 'sagemaker_execution_role': 'arn:aws:sts::015469603702:assumed-role/fmbench-role/SageMaker', 'bucket': 'sagemaker-fmbench-write-015469603702'}, 'dir_paths': {'data_prefix': 'data', 'prompts_prefix': 'prompts', 'all_prompts_file': 'all_prompts.csv', 'metrics_dir': 'metrics', 'models_dir': 'models', 'metadata_dir': 'metadata'}, 's3_read_data': {'read_bucket': 'sagemaker-fmbench-read-015469603702', 'scripts_prefix': 'scripts', 'script_files': ['hf_token.txt'], 'source_data_prefix': 'source_data', 'tokenizer_prefix': 'tokenizer', 'prompt_template_dir': 'prompt_template', 'prompt_template_file': 'prompt_template.txt'}, 'run_steps': {'0_setup.ipynb': True, '1_generate_data.ipynb': True, '2_deploy_model.ipynb': True, '3_run_inference.ipynb': True, '4_model_metric_analysis.ipynb': True,

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


CustomTokenizer, based on HF transformers


#### Pygmentize globals.py to view and use any of the globally initialized variables 

#### Set up a logger to log all messages while the code runs

In [3]:
logging.basicConfig(format='[%(asctime)s] p%(process)s {%(filename)s:%(lineno)d} %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
## config.yml file contains information that is used across this benchmarking environment, 
## such as information about the aws account, prompts, payloads to be used for invocations
config = load_config(CONFIG_FILE)
logger.info(json.dumps(config, indent=2))

[2024-03-20 01:23:52,656] p30320 {2001288519.py:4} INFO - {
  "general": {
    "name": "distilbert-base-uncased-v1",
    "model_name": "distilbert-base-uncased"
  },
  "aws": {
    "region": "us-east-1",
    "sagemaker_execution_role": "arn:aws:sts::015469603702:assumed-role/fmbench-role/SageMaker",
    "bucket": "sagemaker-fmbench-write-015469603702"
  },
  "dir_paths": {
    "data_prefix": "data",
    "prompts_prefix": "prompts",
    "all_prompts_file": "all_prompts.csv",
    "metrics_dir": "metrics",
    "models_dir": "models",
    "metadata_dir": "metadata"
  },
  "s3_read_data": {
    "read_bucket": "sagemaker-fmbench-read-015469603702",
    "scripts_prefix": "scripts",
    "script_files": [
      "hf_token.txt"
    ],
    "source_data_prefix": "source_data",
    "tokenizer_prefix": "tokenizer",
    "prompt_template_dir": "prompt_template",
    "prompt_template_file": "prompt_template.txt"
  },
  "run_steps": {
    "0_setup.ipynb": true,
    "1_generate_data.ipynb": true,
    "2_d

#### Define the file path for the prompt template

In [5]:
s3_file_path = "/".join([config['s3_read_data']['prompt_template_dir'],
                         config['s3_read_data']['prompt_template_file']])

## download the file from s3 else check locally and use that version
prompt_template_from_s3: str = read_from_s3(config['s3_read_data']['read_bucket'], s3_file_path)

if prompt_template_from_s3 is None:
    prompt_template = globals.PROMPT_TEMPLATE
    logger.info(f"Using the default local prompt template --> {prompt_template}")
else:
    prompt_template = prompt_template_from_s3
    logger.info(f"Using the prompt template from S3 --> {prompt_template}")
prompt_template = prompt_template.strip()

# Calculate the number of tokens in the prompt template
prompt_template_keys = config['datasets']['prompt_template_keys']
args = {}
for k in prompt_template_keys:
    args[k] = ""
empty_prompt_template = prompt_template.format(**args)
logger.info(f"empty prompt template = \"{empty_prompt_template}\"")
empty_prompt_len_in_tokens = count_tokens(empty_prompt_template)

# Log the number of tokens
logger.info(f"prompt template length={empty_prompt_len_in_tokens} tokens")

[2024-03-20 01:23:52,733] p30320 {3750880026.py:12} INFO - Using the prompt template from S3 --> {text}
[2024-03-20 01:23:52,734] p30320 {3750880026.py:21} INFO - empty prompt template = ""
[2024-03-20 01:23:52,735] p30320 {3750880026.py:25} INFO - prompt template length=1 tokens


In [6]:
def list_files():
    response = s3_client.list_objects_v2(Bucket=config['s3_read_data']['read_bucket'], Prefix=config['s3_read_data']['source_data_prefix'])
    return [obj['Key'] for obj in response['Contents']]

# List all files in the bucket and prefix
s3_files = list_files()
logger.info(f"s3 paths of the data set -> {s3_files}")

# Log the files you're going to read
logger.info(f"dataset files = {s3_files}")

# Read and concatenate DataFrames
jsonl_files = [file_key for file_key in s3_files if file_key.endswith('.jsonl')]

# Read and concatenate only the .jsonl files
df = pd.concat([pd.read_json(io.BytesIO(s3_client.get_object(Bucket=config['s3_read_data']['read_bucket'], Key=file_key)['Body'].read()), lines=True) 
                for file_key in jsonl_files])

# Log the source of the dataset and its shape
logger.info(f"dataset read from {s3_files}\nhas shape {df.shape}")

[2024-03-20 01:23:52,813] p30320 {490214810.py:7} INFO - s3 paths of the data set -> ['source_data/banking77.jsonl', 'source_data1/banking77.jsonl']
[2024-03-20 01:23:52,814] p30320 {490214810.py:10} INFO - dataset files = ['source_data/banking77.jsonl', 'source_data1/banking77.jsonl']
[2024-03-20 01:23:53,018] p30320 {490214810.py:20} INFO - dataset read from ['source_data/banking77.jsonl', 'source_data1/banking77.jsonl']
has shape (20006, 2)


#### View a portion of the df to view inputs, contexts, and more information on the data

In [7]:
df.head()

,text,label
0,I am still waiting on my card?,11
1,What can I do if my card still hasn't arrived ...,11
2,I have been waiting over a week. Is the card s...,11
3,Can I track my card while it is in the process...,11
4,"How do I know if I will get my card, or if it ...",11


#### Display basic statistics on the existing dataset: including count, mean, std, min, etc.

In [8]:
logger.info(f"distribution of the length field in the dataset is as follows ->\n{df.describe()}")

[2024-03-20 01:23:53,060] p30320 {1912450148.py:1} INFO - distribution of the length field in the dataset is as follows ->
              label
count  20006.000000
mean      37.651704
std       22.390456
min        0.000000
25%       18.000000
50%       37.000000
75%       57.000000
max       76.000000


### Convert the dataset elements into prompts as payloads for inference purposes

Now, we will focus on converting the existing data within our datasets, and extract the information to convert it into prompts to be able to send to our deployed model endpoints during the process of testing and benchmarking for results and various metrics

In [9]:
%%time
df['prompt'] = df.apply(lambda row: process_item(row, config['datasets']['prompt_template_keys'], prompt_template), axis=1)
df['prompt_len'] = df.prompt.map(lambda x: x['prompt_len'])

CPU times: user 2.52 s, sys: 9.98 ms, total: 2.53 s
Wall time: 3.15 s


In [10]:
# Convert DataFrame to a CSV format string
csv_buffer = io.StringIO()
df.to_csv(csv_buffer, index=False)
csv_data = csv_buffer.getvalue()
all_prompts_file = config['dir_paths']['all_prompts_file']

# Write to S3 using the write_to_s3 function
write_to_s3(csv_data, config['aws']['bucket'], DATA_DIR, config['dir_paths']['prompts_prefix'], all_prompts_file)

# Log where the prompts are saved
logger.info(f"all prompts dataframe of shape {df.shape} saved to s3://{config['aws']['bucket']}/{DATA_DIR}/{os.path.join(config['dir_paths']['prompts_prefix'], all_prompts_file)}")

[2024-03-20 01:23:56,681] p30320 {1138732359.py:11} INFO - all prompts dataframe of shape (20006, 4) saved to s3://sagemaker-fmbench-write-015469603702/distilbert-base-uncased-v1-SageMaker/data/prompts/all_prompts.csv


In [11]:
## View some of the prompts 
df.head()

,text,label,prompt,prompt_len
0,I am still waiting on my card?,11,"{'text': 'I am still waiting on my card?', 'te...",9
1,What can I do if my card still hasn't arrived ...,11,{'text': 'What can I do if my card still hasn'...,18
2,I have been waiting over a week. Is the card s...,11,{'text': 'I have been waiting over a week. Is ...,15
3,Can I track my card while it is in the process...,11,{'text': 'Can I track my card while it is in t...,15
4,"How do I know if I will get my card, or if it ...",11,"{'text': 'How do I know if I will get my card,...",18


### Convert Prompts into Payloads for inference purposes
------
Now we will prepare data for model inference. It involves converting prompts, created and stored in a specific format, into payloads for inference. We will utilize the prompt file for our model and incorporate the prompt into a payload using that. 

These payloads are tailored to the needs of deployed model endpoints. The conversion considers prompt sizes and specific configurations to further make our benchmarking more detailed and comprehensive. 

The goal is to have a set of well-formatted and parameterized payload requests of various sizes ready to be sent to the model endpoints for inference, with the responses to be used for further analysis

In [12]:
# Function to construct a single request payload based on row prompt data and configuration
def construct_request_payload(row, config: Dict) -> Dict:
    
    # Deep copy inference parameters from the config.yml file - feel free to change this based on the model type you are using
    parameters = copy.deepcopy(config['inference_parameters'])
    truncate = parameters.get('truncate', None)
    if truncate == TRUNCATE_POLICY.AT_PROMPT_TOKEN_LENGTH:
        parameters['truncate'] = row['prompt_len']
        
    # Return the constructed payload
    return dict(inputs=row['prompt']['prompt'], parameters=parameters)

# Function to create a dataset payload files from the given dataset file we have
def create_dataset_payload_file(df: pd.DataFrame, dataset_info: Dict, config: Dict) -> str:
    
    # First, log the dataset existing information
    logger.info(f"going to create a payload file as dataset_info={json.dumps(dataset_info, indent=2)}")
    
    # Filter the DataFrame based on prompt length and language given below for constructing payloads of various sizes
    df['prompt_len_in_range'] = df.prompt.map(lambda x: x['prompt_len'] >= dataset_info['min_length_in_tokens'] and \
                                                        x['prompt_len'] <= dataset_info['max_length_in_tokens'])
    
    # select prompts between pre-configured threshold lengths and are in the selected language
    if 'language' in df.columns:
        df_filtered = df[(df.language == dataset_info['language']) & (df.prompt_len_in_range)]
    else:
        df_filtered = df[df.prompt_len_in_range]
        
    logger.info(f"after filtering for {json.dumps(dataset_info, indent=2)}, shape of dataframe is {df_filtered.shape}")
    if df_filtered.shape[0] == 0:
        logger.error(f"did not find any prompts in the dataframe that matched the filtering criteria, exiting")
        return None
    # df_filtered.head()

    # Here, we construct request payloads for each row in the filtered DataFrame
    df_filtered['request'] = df_filtered.apply(lambda r: construct_request_payload(r, config), axis=1)
    logger.info(f"payload request entry looks like this -> {json.dumps(df_filtered['request'].iloc[0], indent=2)}")
    
     # Convert the 'request' column of the filtered DataFrame to a JSON Lines string
    json_lines_str = df_filtered['request'].to_json(orient='records', lines=True)
    
    
    lang = dataset_info['language']
    min_len = dataset_info['min_length_in_tokens']
    max_len = dataset_info['max_length_in_tokens']
    file_name = dataset_info['payload_file'].format(lang=lang, min=min_len, max=max_len)

    prompts_path = os.path.join(DATA_DIR, config['dir_paths']['prompts_prefix'])

    ## defining the s3_path these prompts will go to
    s3_file_path = os.path.join(prompts_path, file_name)

    # Write the JSON Lines string to S3
    # get the bucket name, config vars from config file
    write_to_s3(json_lines_str, config['aws']['bucket'], DATA_DIR, config['dir_paths']['prompts_prefix'], file_name)

    logger.info(f"dataset of different payload file structures saved to s3://{config['aws']['bucket']}/{s3_file_path}")
    return f"s3://{config['aws']['bucket']}/{s3_file_path}"

In [13]:
items = ((df, d, config) for d in config['datasets']['filters'])

# This results in the creation of payload files for each dataset
paths: List = list(itertools.starmap(create_dataset_payload_file, items))

[2024-03-20 01:23:56,708] p30320 {2925969590.py:17} INFO - going to create a payload file as dataset_info={
  "language": "en",
  "min_length_in_tokens": 1,
  "max_length_in_tokens": 500,
  "payload_file": "payload_en_1-500.jsonl"
}
[2024-03-20 01:23:56,719] p30320 {2925969590.py:29} INFO - after filtering for {
  "language": "en",
  "min_length_in_tokens": 1,
  "max_length_in_tokens": 500,
  "payload_file": "payload_en_1-500.jsonl"
}, shape of dataframe is (20006, 5)
[2024-03-20 01:23:56,882] p30320 {2925969590.py:37} INFO - payload request entry looks like this -> {
  "inputs": "I am still waiting on my card?",
  "parameters": {
    "ContentType": "application/x-text",
    "Accept": "application/json;verbose"
  }
}
[2024-03-20 01:23:57,136] p30320 {2925969590.py:57} INFO - dataset of different payload file structures saved to s3://sagemaker-fmbench-write-015469603702/distilbert-base-uncased-v1-SageMaker/data/prompts/payload_en_1-500.jsonl
[2024-03-20 01:23:57,141] p30320 {2925969590.

In [14]:
print("\n".join([p for p in paths if p]))

s3://sagemaker-fmbench-write-015469603702/distilbert-base-uncased-v1-SageMaker/data/prompts/payload_en_1-500.jsonl
s3://sagemaker-fmbench-write-015469603702/distilbert-base-uncased-v1-SageMaker/data/prompts/payload_en_100-250.jsonl
s3://sagemaker-fmbench-write-015469603702/distilbert-base-uncased-v1-SageMaker/data/prompts/payload_en_10-20.jsonl
s3://sagemaker-fmbench-write-015469603702/distilbert-base-uncased-v1-SageMaker/data/prompts/payload_en_50-150.jsonl
